In [1]:
import nbimporter
import numpy as np
import matplotlib.pyplot as plt

import gas_water
import liquid_water
import solid_material

import flux
import balance_equation
import property_conversion  as prop
import moisture_conversion as mc

Importing Jupyter notebook from gas_water.ipynb
Importing Jupyter notebook from moisture_conversion.ipynb
Importing Jupyter notebook from vapour_pressure.ipynb
Importing Jupyter notebook from liquid_water.ipynb
Importing Jupyter notebook from property_conversion.ipynb
Importing Jupyter notebook from solid_material.ipynb
Importing Jupyter notebook from flux.ipynb
Importing Jupyter notebook from balance_equation.ipynb


## 1次元モデルの差分計算をするためのモジュール  


### クラス：Cell( i )  
差分法におけるコントロールボリューム（CV）に相当する構造体。  
固相・液相・気相の情報から構成され、CVの大きさ、位置などの情報を有する。  
示量変数はCellクラスに内包され、示強変数はそれぞれの相情報に保持される。

- cell情報  
    i：セルの位置    
    dx：セルの大きさ    
    
    - 熱・水分・物質移動係数
        各種移動係数は後述のcellの構成情報に影響を受けるためcellクラスが保有する

- 気相情報(Gas)  
    - 水蒸気(gas_water)  
        temp：温度[K]  
        rh：相対湿度[-]  
        miu：水分化学ポテンシャル[J/kg]  
        pv：水蒸気圧[Pa]  
    - 空気（未実装）


- 液相情報(Liquid)  
    - 純水(liquid_water)  
        temp：温度[K]  
        miu：水分化学ポテンシャル[J/kg]  
        phi：含水率[-]  
        pl：水圧[Pa]  
    - 塩溶液（未実装）  
        

- 固相情報(Solid)  
    - 材料骨格部(solid_material)  
        material：材料の名称  
        crow：熱容量（比熱×密度）  
        ※以下は液相・気相の影響を受ける  
        lam：熱伝達率  
        phi：含水率  
        dphi：含水率の水分化学ポテンシャル微分  
        ldp：透湿率  
        ldml：水分化学ポテンシャル勾配に対する液相水分伝導率  
    - 氷（未実装）   
    - 塩（未実装）  


In [2]:
class  Cell():
    def __init__( self, i ):
        self.position = i
    
    # cell information
    # Length of cell
    def set_dx( self, initial = 0.0 ):
        self.dx = initial 
    
    # Gas condition（気相）
    def set_Gas_Water_BasedMiu( self, temp = 0.0, miu = 0.0 ):
        self.vapour = gas_water.MiuBasedMoisture( temp, miu )
        
    def set_Gas_Water_BasedRH( self, temp = 0.0, rh = 0.0 ):
        self.vapour = gas_water.RHBasedMoisture( temp, rh )
        
    def set_Gas_Water_BasedPV( self, temp = 0.0, pv = 0.0 ):
        self.vapour = gas_water.PVBasedMoisture( temp, pv )
        
    # Liquid condition（液相）
    def set_Liquid_Water_BasedMiu( self, temp = 0.0, miu = 0.0 ):
        self.water = liquid_water.MiuBasedWater( temp, miu )
        
    def set_Liquid_Water_BasedPhi( self, temp = 0.0, phi = 0.0 ):
        self.water = liquid_water.PhiBasedWater( temp, phi )
        
    # Solid condition（固相）
    def set_Solid_Material( self, temp, material_name ):
        self.material = solid_material.TempBased( temp, material_name )
        
    # Transfer coefficient（セルの有する移動係数）        
    # lam 熱伝導率
    def get_lam(self):
        return prop.Kernel_TransferCoefficient( self.material.name ).getLAM( self.water )
    
    lam = property(get_lam)
        
    # dp 湿気伝導率
    def get_dp(self):
        return prop.Kernel_TransferCoefficient( self.material.name ).getDP( self.water )
    
    dp = property(get_dp)
        
    # ldml 液水の水分伝導率
    def get_ldml(self):
        return prop.Kernel_TransferCoefficient( self.material.name ).getLDML( self.water )

    ldml = property(get_ldml)
    

### 使用例

セルの個数を決定する

In [3]:
number = 5

セルの作成・大きさを与える

In [11]:
wall = [ Cell(i) for i in range(number)]
[ cell.set_dx(0.1) for cell in wall ]

[None, None, None, None, None]

水蒸気の情報（温度20℃、相対湿度60%）を与える

In [17]:
[ cell.set_Gas_Water_BasedRH( temp = 293.15, rh = 0.6 ) for cell in wall ]

[None, None, None, None, None]

材料情報を与える

In [18]:
[ cell.set_Solid_Material( temp = cell.vapour.temp, material_name = 'BentheimerSandstone' ) for cell in wall ]

[None, None, None, None, None]

水蒸気・材料の情報から液水の情報を算出する

In [19]:
[ cell.set_Liquid_Water_BasedMiu( temp = cell.vapour.temp, miu = cell.vapour.miu ) for cell in wall ]
[ cell.water.set_material( material_name = cell.material.name ) for cell in wall ]

[None, None, None, None, None]

#### 値の確認
セル情報の構築は以上となる。セル情報をいくつか引き出しみる。  
例えば液水の含水率は

In [20]:
wall[1].water.phi

Importing Jupyter notebook from bentheimer_sandstone.ipynb
Importing Jupyter notebook from van_genuchten.ipynb


3.2608830802176583e-05

セルの液相水分伝導率は

In [21]:
wall[1].ldml

2.4529155342643628e-23

### クラス：Air( i )  
材料境界などに位置する室空気のクラス。  
Cellクラスと基本概念は同じであるが固相・液相情報は持たず気相情報のみを持つものとする。  
また、Cellクラス同様に空間の容積などの示量変数はAirクラスに内包され、示強変数はそれぞれの相情報に保持される。  
現状、室空気は温度と水分情報しか持たないことが多いため、gas_waterのクラスのみを用いることで代用した。

- Air情報     
    i : Airの名称（基本的には何でもよい）  
    vol：Airの容積[m3]
    
    
    - 熱・水分・物質伝達係数

- 気相情報(Gas)  
    - 水蒸気(gas_water)  
        temp：温度[K]  
        rh：相対湿度[-]  
        miu：水分化学ポテンシャル[J/kg]  
        pv：水蒸気圧[Pa]  
    - 空気（未実装）


In [ ]:
class  Air():
    def __init__( self, i ):
        self.position = i
    
    # cell information
    # Length of cell
    def set_volume( self, volume = 0.0 ):
        self.vol = volume 
    
    # Gas condition（気相）
    def set_Air_BasedMiu( self, temp = 0.0, miu = 0.0 ):
        self.air = gas_water.MiuBasedMoisture( temp, miu )
        
    def set_Air_BasedRH( self, temp = 0.0, rh = 0.0 ):
        self.air = gas_water.RHBasedMoisture( temp, rh )
        
    def set_Air_BasedPV( self, temp = 0.0, pv = 0.0 ):
        self.air = gas_water.PVBasedMoisture( temp, pv )
        
    # Transfer coefficient（セルの有する移動係数）
    def set_HeatTransferCoefficient( self, alpha ):
        self.alpha = alpha
        
    def set_MoistureTransferCoefficient( self, aldm ):
        self.aldm = aldm
        

### 使用例
基本はCellクラスと同じなので

In [ ]:
# 使用例
a = Air('室空気')

In [ ]:
a.set_volume(100.0)
a.set_Air_BasedRH( temp = 283.15, rh = 0.8 ) 
a.set_HeatTransferCoefficient( alpha = 9.3 )
a.set_MoistureTransferCoefficient( aldm = 9.3e-10 )

#### 値の確認

In [ ]:
a.air.pv

## 流量計算  

Fluxモジュールは2つのCellを与えることでその間の熱・水分流を計算する。  
例えばcell[1]とcell[2]間の流量は、

In [ ]:
flux1 = flux.RobinBoundaryCondition( cell_mns = c[1], cell_pls = c[2], nx =0.0 )

熱流の計算結果は、

In [ ]:
flux1.qs

水蒸気流の計算結果は、

In [ ]:
flux1.jv

液水流の計算結果は、

In [ ]:
flux1.jl

同様に空気層との流量計算もでき、

In [ ]:
flux2 = flux.RobinBoundaryCondition( cell_mns = a, cell_pls = c[0], nx =0.0 )

熱流

In [ ]:
flux2.qs

水蒸気流

In [ ]:
flux2.jv

In [ ]:
flux2.jl

これを一括計算しようとすると以下のようになる。  
例えば  

空気　―　壁（cell[0] ～　cell[end]）　―　空気  

のような状況を想定する。  

※wall = c とするとmutableなオブジェクトとしてコピーされるのでスライスで対処

In [ ]:
wall1 = c[:]
wall1.insert(0,a)
wall1.insert(len(wall1),a)

wall1のコンストラクタを確認してみる。まずは空気層のある場所について、

In [ ]:
[ hasattr(wall1[i], 'air') for i in range(len(wall1)) ]

次に多孔質材料の有無、

In [ ]:
[ hasattr(wall1[i], 'material') for i in range(len(wall1)) ]


流量を計算してみる。  
※第二種と第三種境界条件の複合条件とすると、


In [ ]:
flux3 = [ flux.CombinationBoundaryCondition( cell_mns = wall1[i], cell_pls = wall1[i+1], nx =0.0 ) for i in range(len(wall1)-1) ]

#### 値の確認  
熱流

In [ ]:
[ flux3[i].qs for i in range(len(flux3)) ]

第二種境界条件として境界に日射量10[W/m2]を足してみる。

In [ ]:
flux3[0].set_qs( 10.0 )

In [ ]:
[ flux3[i].qs for i in range(len(flux3)) ]

水蒸気流

In [ ]:
[ flux3[i].jv for i in range(len(flux3)) ]

液相水分流

In [ ]:
[ flux3[i].jl for i in range(len(flux3)) ]

## 熱・水分の収支計算

収支計算モジュールはcellとcell両端に流出入するflux、時間刻みdtを入力することで計算できる。  
ここでは上述のwall1とflux3を使うことで以下のように計算する。  
まずはwall1の表面[1]について

In [ ]:
new = balance_equation.MiuBased( cell = wall1[1], flux_mns = flux3[0], flux_pls = flux3[1], dt = 0.01 )

#### 値の確認  
温度の計算結果は、

In [ ]:
new.temp

水分化学ポテンシャルの計算結果は、

In [ ]:
new.miu

壁全体（境界の空気は除く）に適応すると、

In [ ]:
new1 = [ balance_equation.MiuBased( cell = wall1[i+1], flux_mns = flux3[i], flux_pls = flux3[i+1], dt = 0.01 ) for i in range(len(wall1)-2)]

温度は、

In [ ]:
[ new1[i].temp for i in range(len(new1))]

水分化学ポテンシャルは、

In [ ]:
[ new1[i].miu for i in range(len(new1))]

あとは計算結果をcellクラスに代入しループ計算をすれば完成。  
以上の計算を簡単にまとめると、単一材料系における計算は、

#### セルの構成

In [ ]:
def set_cell( number, length ):
    cell = [ Cell(i) for i in range(number)]
    [ cell[i].set_dx(0.1) for i in range(number)]
    return cell

#### 初期温湿度、材料の設定

In [ ]:
def set_condition( cell, temp, rh, material ):
    [ cell[i].set_Gas_Water_BasedRH( temp, rh ) for i in range(len(cell)) ]
    [ cell[i].set_Solid_Material( temp , material ) for i in range(len(cell))]
    [ cell[i].set_Liquid_Water_BasedMiu( temp , miu = cell[i].vapour.miu ) for i in range(len(cell)) ]
    [ cell[i].water.set_material( material ) for i in range(len(cell)) ]

#### 室空気のセット

In [ ]:
def set_air( name, volume, temp, rh, alpha, aldm ):
    air = Air(name)
    air.set_volume(temp)
    air.set_Air_BasedRH( temp, rh ) 
    air.set_HeatTransferCoefficient( alpha )
    air.set_MoistureTransferCoefficient( aldm )
    return air

#### 流量の計算

In [ ]:
def cal_flux( air_in, cell, air_out, nx ):
    wall = cell[:]
    wall.insert(0,air_in)
    wall.insert(len(wall1),air_out)
    return [ flux.CombinationBoundaryCondition( wall[i], wall[i+1], nx ) for i in range(len(wall)-1) ]

#### 収支計算

In [ ]:
def new_value( cell, flux, dt ):
    return [ balance_equation.MiuBased( cell[i], flux[i], flux[i+1], dt ) for i in range(len(cell)) ]

#### 新しい温湿度のセット

In [ ]:
def set_temp_miu( cell, new_value ):
    [ cell[i].vapour.set_temp(new_value[i].temp) for i in range(len(cell)) ]
    [ cell[i].water.set_temp(new_value[i].temp) for i in range(len(cell)) ]
    [ cell[i].material.set_temp(new_value[i].temp) for i in range(len(cell)) ]
    [ cell[i].vapour.set_miu(new_value[i].miu) for i in range(len(cell)) ]
    [ cell[i].water.set_miu(new_value[i].miu) for i in range(len(cell)) ]

### 使用例

cellのセット

In [ ]:
cell_test = set_cell( number = 10, length = 0.1 )
set_condition( cell = cell_test, temp = 293.15, rh = 0.7, material = 'BentheimerSandstone' )

境界（空気）の温湿度のセット

In [ ]:
inner_air = set_air( "室空気", volume = 100.0 , temp = 288.15, rh = 0.7, alpha = 9.0,  aldm = 3.0e-10 )
outer_air = set_air( "外気",   volume = 100.0 , temp = 298.15, rh = 0.65,alpha = 23.0, aldm = 9.3e-10 )

流量計算及び収支計算並びに温湿度のセット

In [ ]:
for i in range(1000):
    flux_test  = cal_flux ( air_in = inner_air, cell = cell_test, air_out = outer_air , nx = 0.0 )
    nvalue_test = new_value( cell = cell_test  , flux = flux_test, dt = 0.1 )
    set_temp_miu( cell = cell_test, new_value = nvalue_test )

In [ ]:
[ cell_test[i].material.temp for i in range(len(cell_test)) ]